In [1]:
from random import random
from random import seed
from math import exp

In [2]:
def initialized(n_inputs,n_hidden,n_outputs):
    network=list()
    hidden_layer=[{'weights':[random()for i in range(n_inputs+1)]}for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer=[{'weights':[random()for i in range(n_hidden+1)]}for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network=initialized(2,1,2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [3]:
def activate(weights,inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
        activation +=weights[i]*inputs[i]
    return activation

def transfer(activation):
    return 1/(1+exp(-activation))


def forward(network,row):
    inputs=row
    for layer in network:
        new_input=[]
        for neuron in layer:
            activation=activate(neuron['weights'],inputs)
            neuron['outputs']=transfer(activation)
            new_input.append(neuron['outputs'])
        inputs=new_input
    return inputs

network=[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row=[1,0,None]
outputs=forward(network,row)
print(outputs)

[0.6629970129852887, 0.7253160725279748]


In [4]:
def tranfer_deri(outputs):
    return outputs*(1-outputs)

def backward(network,expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error=0
                for neuron in network[i+1]:
                    error +=(neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(neuron['outputs']-expected[j])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*tranfer_deri(neuron['outputs'])

network=[[{'outputs':0.71,'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'outputs':0.65,'weights': [0.2550690257394217, 0.49543508709194095]}, {'outputs':0.62,'weights': [0.4494910647887381, 0.651592972722763]}]]
expected=[0,1]
backward(network,expected)
for layer in network:
    print(layer)     

[{'outputs': 0.71, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005196306262542487}]
[{'outputs': 0.65, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14787499999999998}, {'outputs': 0.62, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.089528}]


In [5]:
def update(network,row,lr):
    for i in range(len(network)):
        inputs=row[:-1]
        if i != 0:
            inputs=[neuron['outputs']for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -=lr*neuron['delta']*inputs[j]
            neuron['weights'][-1] -=lr*neuron['delta']

def train_network(network,train,lr,ep,n_outputs):
    for epoch in range(ep):
        sum_error=0
        for row in train:
            outputs=forward(network,row)
            expected=[0 for i in range(n_outputs)]
            expected[row[-1]]=1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward(network,expected)
            update(network,row,lr)
        print(f"epoch {epoch} lr {lr} Mse :{sum_error}")


seed(1)
dataset=[[0,0,0],[0,1,1],[1,0,1],[1,1,0]]
n_inputs=len(dataset[0])-1
n_outputs=len(set([row[-1]for row in dataset]))
initialized(n_inputs,2,n_outputs)
train_network(network,dataset,0.5,50000,n_outputs)

epoch 0 lr 0.5 Mse :2.366190129625063
epoch 1 lr 0.5 Mse :2.300289382940681
epoch 2 lr 0.5 Mse :2.2475342991018366
epoch 3 lr 0.5 Mse :2.206801659524181
epoch 4 lr 0.5 Mse :2.1762951214202437
epoch 5 lr 0.5 Mse :2.1539993044950716
epoch 6 lr 0.5 Mse :2.138007805556958
epoch 7 lr 0.5 Mse :2.1266964688831695
epoch 8 lr 0.5 Mse :2.1187745351542393
epoch 9 lr 0.5 Mse :2.1132637705460127
epoch 10 lr 0.5 Mse :2.109446637736131
epoch 11 lr 0.5 Mse :2.106808545164957
epoch 12 lr 0.5 Mse :2.1049861267144783
epoch 13 lr 0.5 Mse :2.1037255232214216
epoch 14 lr 0.5 Mse :2.1028506770802005
epoch 15 lr 0.5 Mse :2.1022401035778495
epoch 16 lr 0.5 Mse :2.101810267606288
epoch 17 lr 0.5 Mse :2.1015038692432126
epoch 18 lr 0.5 Mse :2.1012816727407557
epoch 19 lr 0.5 Mse :2.101116846147625
epoch 20 lr 0.5 Mse :2.1009910583051834
epoch 21 lr 0.5 Mse :2.1008917960756732
epoch 22 lr 0.5 Mse :2.100810524277676
epoch 23 lr 0.5 Mse :2.100741425494486
epoch 24 lr 0.5 Mse :2.100680537917561
epoch 25 lr 0.5 Mse :

In [6]:
def predict(network,row):
    outputs=forward(network,row)
    return outputs.index(max(outputs))

for row in dataset:
    prediction=predict(network,row)
    print(f"Actual {row[-1]} Predicted {prediction}")

Actual 0 Predicted 0
Actual 1 Predicted 1
Actual 1 Predicted 1
Actual 0 Predicted 1
